In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [4]:
a = 3.4
# lambdas = a/np.array([0.22,0.88]) 
lambdas = np.array([7,2])/0.8


In [ ]:
# folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5"
folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250124"
project_name = "01_24_2025 H5 LSU Transmission res 10"
postprocess_results = []
runtime_ps = 10e-12
min_steps_per_lambda = 10
cuts = [1]
for direction in ["z"]: 
    for f,filename in enumerate(os.listdir(folder_path)):
     for cut in cuts:
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 

           
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                           monitors=["flux"], freqs=280, 
                                           cut_condition=cut, source="planewave", absorbers=120, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}"
                                           )
            
        
        file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value}\z_incidence\{structure_1.sim_name}.txt"
        print(file_desc)
        if os.path.exists(file_desc):
            print("Exist!")
        else:
           print("Creating...")
           structure_1.run_sim(run_free=False,load=False,add_ref=True,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value}")

        del structure_1

            
            # structure_1.plot_sim_layout()
